In [ ]:
#calculate absorption spectrum 

from pyscf.pbc import gto, dft
import numpy as np
import matplotlib.pyplot as plt

# Define the graphene unit cell
cell = gto.Cell()
cell.atom = '''
C 0.0 0.0 0.0
C 0.0 1.23 0.0
'''
cell.a = [[2.46, 0, 0], [0, 2.46, 0], [0, 0, 20.0]]  # Lattice + vacuum for graphene
cell.basis = 'gth-szv'  # Basis set
cell.pseudo = 'gth-pade'  # Pseudopotentials
cell.build()

# Define k-point sampling (mesh size)
kmesh = [4, 4, 1]  # Sampling along the x and y axes
kpts = cell.make_kpts(kmesh)  # Create k-points grid

mf = dft.KRKS(cell, kpts)  # Use KRKS to handle k-points
mf.xc = 'pbe'  # Choose the exchange-correlation functional (e.g., PBE)
mf.kernel()  # Perform the DFT calculation

# Extract the eigenvalues and the transition dipoles
eigenvalues = np.array(mf.mo_energy) 
# Define the number of occupied and unoccupied states
n_occ = cell.nao-2  # Number of occupied states 
n_k = len(kpts)   # Number of k-points

# Separate occupied and unoccupied states
occupied_states = eigenvalues[:, :n_occ]  # Eigenvalues for occupied states 
unoccupied_states = eigenvalues[:, n_occ:]  # Eigenvalues for unoccupied states 
# Initialize an array for the absorption spectrum
absorption_spectrum = []

# Define the energy range for the absorption spectrum
emin = 0.00000000001  # Energy range from the minimum of occupied states
emax = np.max(unoccupied_states) - np.min(occupied_states)# Energy range from the maximum of unoccupied states
#energy_grid = np.linspace(emin, emax, 1000)  # Energy grid
energy_grid = np.linspace(0.00000001, np.max(eigenvalues)-np.min(eigenvalues), 1000)

# Define Gaussian broadening (standard deviation)
sigma = 0.1
absorption_spectrum = np.zeros_like(energy_grid)

# Loop over all occupied and unoccupied states for all k-points
for k in range(n_k):  # Loop over k-points
    for i in range(n_occ):  # Loop over occupied states
        for j in range(n_occ, eigenvalues.shape[1]):  # Loop over unoccupied states
            # Transition energy (difference between occupied and unoccupied states)
            transition_energy = unoccupied_states[k, j - n_occ] - occupied_states[k, i]
            
            # Apply Gaussian broadening
            absorption_spectrum += np.exp(-0.5 * ((energy_grid - transition_energy) / sigma) ** 2) / (sigma * np.sqrt(2 * np.pi))

# Normalize the absorption spectrum
absorption_spectrum /= np.max(absorption_spectrum)
wavelength = 1239.841984/energy_grid
# Plot the absorption spectrum
#plt.plot(energy_grid, absorption_spectrum)
#plt.xlabel('Energy (eV)')
plt.plot(wavelength, absorption_spectrum)
plt.xlabel('Wavelength (nm)')
plt.ylabel('Absorption Spectrum (arbitrary units)')
plt.title('Absorption Spectrum of Graphene')
plt.xlim(0,5)
plt.show()
